In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import pickle
import requests
import os
import pathlib

In [15]:
driver = webdriver.Chrome('../../chromedriver')

In [28]:
whitelist = ['minimal', 'landscape', 'abstract']
blacklist = ['phone']
WEBSITE_PAUSE_TIME = 1
IMAGE_PAUSE_TIME = 0.5
SCROLL_PAUSE_TIME = 1
page_limit = 200*10

In [29]:
image_urls=[]
viewed_pages = set()
image_categories={}

queue = [
    "https://wallpaperaccess.com/minimal-landscape",
    "https://wallpaperaccess.com/sunset-minimal",
    "https://wallpaperaccess.com/minimal-city",
    "https://wallpaperaccess.com/minimal-phone",
    "https://wallpaperaccess.com/modern-minimal"
]

In [11]:
# DOESN'T WORK

try:
    with open('/data1/wallpaperAccessPhotos.pkl', 'rb') as f:
        image_urls = pickle.load(f)
        print(len(image_urls))
        
    with open('/data1/wallpaperAccessPhotoCategories.pkl', 'rb') as f:
        image_categories = pickle.load(f)
        print(len(image_categories))
        
        viewed_pages = set(image_categories.keys())
        print(len(viewed_pages))
        
    print('Avg imgs per cat', len(image_urls)/len(viewed_pages))

except Exception:
    print('Does not exist')

21976
19230
19230
Avg imgs per cat 1.1427977119084762


In [18]:
def scroll_to_bottom(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [30]:
for i in tqdm(range(page_limit)):
    
    if len(queue) <= 0:
        break
        
    url = queue.pop(0)
    
    if url in viewed_pages:
        continue
    
    driver.get(url)
    
    time.sleep(WEBSITE_PAUSE_TIME)
    scroll_to_bottom(driver)
    time.sleep(WEBSITE_PAUSE_TIME)
    
    imgs = driver.find_elements(By.XPATH, '//img[@class="thumb ads_popup"]')
    
    image_category = url.split('/')[-1]
    
    for img in imgs:
        img_url = img.get_attribute('src')
        image_urls.append(img_url)
        
        if img_url not in image_categories:
            image_categories[img_url] = []
        image_categories[img_url].append(url)
        
        img_id = img_url.split('/')[-1]
        path = pathlib.Path('/data1', 'Raw', 'WallpaperAccess', image_category, img_id)
        path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(path,'wb') as f:
            f.write(requests.get(img_url).content)
            f.close()
        
        time.sleep(IMAGE_PAUSE_TIME)
        
        
    print(url, len(imgs))
    print(len(image_urls))
        
    related_wallpapers = driver.find_elements(By.XPATH, '//div[@class="column collection_thumb"]/a')
    
    for link in related_wallpapers:
        wp_link = link.get_attribute('href')
        eval_val = 0
        
        for whitelist_word in whitelist:
            if whitelist_word in wp_link:
                eval_val += 1
        for blacklist_word in blacklist:
            if blacklist_word in wp_link:
                eval_val -= 1
        if eval_val > 0:
            queue.insert(0, wp_link)
        else:
            queue.append(wp_link)
        
    viewed_pages.add(url)
    
    if (i%50) == 0:
        with open('/data1/wallpaperAccessPhotos.pkl', 'wb') as f:
            pickle.dump(image_urls, f)

        with open('/data1/wallpaperAccessPhotoCategories.pkl', 'wb') as f:
            pickle.dump(image_categories, f)
            
    time.sleep(WEBSITE_PAUSE_TIME)

  0%|          | 0/2000 [00:00<?, ?it/s]

https://wallpaperaccess.com/minimal-landscape 54
54


  0%|          | 1/2000 [00:43<23:55:44, 43.09s/it]

https://wallpaperaccess.com/black-minimal 48
102


  0%|          | 2/2000 [01:22<23:14:40, 41.88s/it]

...

 99%|█████████▉| 1984/2000 [6:57:20<05:06, 19.15s/it]

https://wallpaperaccess.com/black-and-orange-abstract 49
30519


 99%|█████████▉| 1985/2000 [6:58:00<06:20, 25.36s/it]

https://wallpaperaccess.com/white-hd-abstract 53
30572


100%|██████████| 2000/2000 [6:58:42<00:00, 12.56s/it]


In [31]:
with open('/data1/wallpaperAccessPhotos.pkl', 'wb') as f:
    pickle.dump(image_urls, f)

with open('/data1/wallpaperAccessPhotoCategories.pkl', 'wb') as f:
    pickle.dump(image_categories, f)

10000